## Loading and cleaning

In [40]:
import pandas as pd
import numpy as np
import re

In [41]:
f = open('../repository/RAW/FI - Economic Freedom.csv', mode='r', encoding='utf-8')
df = pd.read_csv(f, sep=';', index_col=False, decimal=',')
df = df.dropna(axis=0).convert_dtypes()

In [42]:
ptrn = re.compile(r'^\d\w+\s') # Removing prefixes
rm_prefix = lambda x: ptrn.sub('', x).strip()
df = df.rename(rm_prefix, axis='columns')
df = df.drop(columns=list(el.group(0) for el in map(lambda x: re.match(r'\d.*', x), df.columns) if el))
df = df.drop(columns=list(el.group(0) for el in map(lambda x: re.match(r'data.*', x), df.columns) if el))

## Spliting dataframe

In [43]:
f = open('../repository/DUMP/ISO CODE.csv', 'r')
iso_codes = pd.read_csv(f)
iso_codes['id'] = pd.Series(range(len(iso_codes)), name='id')

In [44]:
df = df.merge(iso_codes.iloc[:, [-1, 0, 2]], left_on="ISO Code 3", right_on="alpha-3")
df.drop(axis=1, columns=df.columns[1:4]).drop(axis=1, columns=df.columns[-1:-5])

,Year,Economic Freedom Summary Index,Rank,Quartile,Government consumption,Transfers and subsidies,Government investment,Top marginal income tax rate,Top marginal income and payroll tax rate,Top marginal tax rate,...,Market openness,Business Permits,Distorton of the business environment,Freedom to enter markets and compete,Area 5 Rank,World Bank Region,"World Bank Current Income Classification, 1990-present",id,name,alpha-3
0,2021,7.6,31,1,7.76,7.13,7.55,9,7,8.0,...,6.81,5.62,6.25,6.23,51,Europe & Central Asia,UM,2,Albania,ALB
1,2020,7.59,32,1,8.03,7.05,7.55,9,7,8.0,...,6.81,5.62,6.25,6.23,73,Europe & Central Asia,UM,2,Albania,ALB
2,2019,7.71,35,1,8.16,7.33,7.55,9,7,8.0,...,6.81,5.62,6.25,6.23,56,Europe & Central Asia,UM,2,Albania,ALB
3,2018,7.69,37,1,8.16,6.74,7.63,9,7,8.0,...,6.81,5.62,6.25,6.23,52,Europe & Central Asia,UM,2,Albania,ALB
4,2017,7.69,38,1,8.08,6.66,8.0,9,7,8.0,...,6.81,6.03,6.25,6.36,46,Europe & Central Asia,UM,2,Albania,ALB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,2019,5.09,157,4,9.21,8.5,4.19,5,5,5.0,...,2.08,7.99,0.0,3.36,153,Sub-Saharan Africa,LM,248,Zimbabwe,ZWE
529,2018,5.36,149,4,8.38,8.5,2.82,4,4,4.0,...,2.08,7.99,0.0,3.36,146,Sub-Saharan Africa,L,248,Zimbabwe,ZWE
530,2017,5.0,153,4,5.46,9.33,0.0,4,4,4.0,...,1.95,7.49,0.0,3.15,141,Sub-Saharan Africa,L,248,Zimbabwe,ZWE
531,2017,6.01,127,4,8.67,8.64,1.23,9,7,8.0,...,2.95,9.43,0.0,4.13,151,Sub-Saharan Africa,L,126,Liberia,LBR


In [45]:
gov_col = [
    "Government consumption",
    "Transfers and subsidies",
    "Government investment",
    "Judicial independence",
    "Impartial courts",
    "Military interference",
    "Legal integrity",
    "Police and crime",
    "Impartial Public Administration",
    "Tax compliance",
]

sg_col = df.columns.difference(gov_col, sort=False)

In [46]:
freedom = df.loc[:, gov_col]
sg = df.loc[:, sg_col]

In [47]:
freedom.to_csv('../repository/DUMP/Economic Freedom/EF - Freedom.csv')
sg.to_csv('../repository/DUMP/Economic Freedom/EF - State Governance.csv')